<a href="https://colab.research.google.com/github/ved3prakas/yolact/blob/master/yolact%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLACT Google Colab Notebook
#Inference on images and video with [YOLACT](https://github.com/dbolya/yolact).

## Google Colab
This notebook is set up to work inside Google Colab, which is a free, Linux-based Jupyter Notebook environment hosted in the cloud.

If you try it outside of Google Colab I'm not sure if it will work. I originally had problems running locally on my Windows machine, which is why I switched to this.

### Runtime Setup
- Go to Runtime > Change Runtime Type
- Choose GPU (TPU won't work)

## YOLACT
YOLACT is a state of the art, real-time, single shot object segmentation algorithm detailed in these papers:  
- [YOLACT: Real-time Instance Segmentation](https://arxiv.org/abs/1904.02689)
- [YOLACT++: Better Real-time Instance Segmentation](https://arxiv.org/abs/1912.06218)

**Big thanks to the authors: Daniel Bolya, Chong Zhou, Fanyi Xiao, Yong Jae Lee!**



In [ ]:
#Mount your google drive here first 
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pwd

/content


In [2]:
# since you are inside */content* you need to go inside MY Drive
%cd /content/drive/My Drive

/content/drive/My Drive


In [ ]:
#since we are inside My Drive

In [3]:
#Make a new Folder inside your My Drive ,let's suppose i make rugby folder
!mkdir -p  rugby

In [4]:
!pwd

/content/drive/My Drive


In [5]:
#go to rugby folder
%cd rugby

/content/drive/My Drive/rugby


In [6]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib

In [7]:
# Downgrade torch to accommodate DCNv2
!pip install torchvision==0.5.0
!pip install torch==1.4.0

     |████████████████████████████████| 4.0MB 5.5MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


## Clone YOLACT from GitHub
Let's get that github repo! If anything in this notebook breaks, check the repo in case they've made updates.

In [8]:
!pwd

/content/drive/My Drive/rugby


In [9]:
# Make sure we're in the top folder
#%cd rugby

# Clone the repo
!git clone https://github.com/dbolya/yolact.git

Cloning into 'yolact'...
remote: Enumerating objects: 2933, done.
remote: Total 2933 (delta 0), reused 0 (delta 0), pack-reused 2933
Receiving objects: 100% (2933/2933), 21.19 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (2000/2000), done.


## DCNv2
The DCNv2 external library is needed for this to work, so we'll build it now.

**IF YOU GET CUDA ERRORS** You need to change the Runtime of this notebook to "GPU" as mentioned at the top of the notebook.

In [10]:
# Change to the right directory
%cd yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

/content/drive/My Drive/rugby/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/content
creating build/temp.linux-x86_64-3.6/content/drive
creating build/temp.linux-x86_64-3.6/content/drive/My Drive
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby/yolact
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby/yolact/external
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby/yolact/external/DCNv2
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby/yolact/external/DCNv2/src
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby/yolact/external/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.6/content/drive/My Drive/rugby/yolact/external/DCNv2/src/cuda
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong 

In [ ]:
#check current working directory

In [12]:
# Get Back to yolact folder 
# just do to get back to main folder(i.e Yolact)
%cd .. 
%cd ..

/content/drive/My Drive/rugby/yolact/external
/content/drive/My Drive/rugby/yolact


In [13]:
#Check whether you are inside yolact folder or not
!pwd

/content/drive/My Drive/rugby/yolact


## Pretrained Weights
In order to run inference, we'll need some pre-trained weights. The creator of the GitHub repo shared them on Google Drive. We're going to use a [helpful tool](https://github.com/chentinghao/download_google_drive) made by [chentinghao](https://github.com/chentinghao) to easily access the Drive file from Colab.

If this stops working, it's probably because the weights moved to a different location. Check the YOLACT github repo to see where they went.

In [14]:

# Clone the repo
!git clone https://github.com/chentinghao/download_google_drive.git

# Create a new directory for the pre-trained weights
!mkdir -p weights

# Download the file
!python ./download_google_drive/download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP weights/yolact_plus_resnet50_54_800000.pth

Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
129MB [00:00, 161MB/s]


# Run Inference on Video
I can't figure out how to get Google Colab to show the output video in the notebook, so I've uploaded the video to YouTube. The mask quality is incredible. There are some false positives because it thinks the rocks are sheep, but we'll give it a pass because it's never been told what a rock is before (COCO dataset doesn't contain rocks) and the rocks do look a bit sheepish.


In [19]:
!pwd

/content/drive/My Drive/rugby/yolact


In [17]:
# Make a new directory for the test videos
!mkdir -p test_video

In [20]:
#give the path of input video 
input_video_path ="test_video/myvideo.mp4"

In [21]:
#Check whether  are you inside yolact folder or not 
#if not make sure you should be inside yolact folder
!pwd 

/content/drive/My Drive/rugby/yolact


In [24]:
#make output folder so that we can store generated output videos
!mkdir -p output_video
output_path = "output_video/my_output.mp4"

In [15]:
#make sure your present working directory is yolact only

**Now we run inference.**

In [25]:
!python eval.py --trained_model=weights/yolact_plus_resnet50_54_800000.pth --config=yolact_plus_resnet50_config --score_threshold=0.15 --top_k=15  --video={input_video_path}:{output_path}

Loading model... Done.
Initializing model... Done.

Processing FPS: 0.00 | Video Playback FPS: 18.45 | Frames in Buffer: 0    


In [ ]:
#TO see the processed video go to you output_video folder  & download to see the resultant video